**Chapter 19 - Training and Deploying Models at Scale**

<table align="left">
  <td>
    <a href="https://colab.research.google.com/github/ageron/handson-ml3/blob/main/19_training_and_deploying_at_scale.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>
  </td>
  <td>
    <a target="_blank" href="https://kaggle.com/kernels/welcome?src=https://github.com/ageron/handson-ml3/blob/main/19_training_and_deploying_at_scale.ipynb"><img src="https://kaggle.com/static/images/open-in-kaggle.svg" /></a>
  </td>
</table>

# Setup

This project requires Python 3.7 or above:

In [1]:
import sys

assert sys.version_info >= (3, 7)

And TensorFlow ≥ 2.8:

In [2]:
from packaging import version
import tensorflow as tf

assert version.parse(tf.__version__) >= version.parse("2.8.0")

2023-05-21 22:08:11.741103: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-05-21 22:08:17.244152: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2023-05-21 22:08:17.244191: I tensorflow/compiler/xla/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.
2023-05-21 22:08:39.610439: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory
2023-

If running on Colab or Kaggle, you need to install the Google AI Platform client library, which will be used later in this notebook. You can ignore the warnings about version incompatibilities.

* **Warning**: On Colab, you must restart the Runtime after the installation, and continue with the next cells.

In [3]:
import sys
if "google.colab" in sys.modules or "kaggle_secrets" in sys.modules:
    %pip install -q -U google-cloud-aiplatform

This chapter discusses how to run or train a model on one or more GPUs, so let's make sure there's at least one, or else issue a warning:

In [4]:
if not tf.config.list_physical_devices('GPU'):
    print("No GPU was detected. Neural nets can be very slow without a GPU.")
    if "google.colab" in sys.modules:
        print("Go to Runtime > Change runtime and select a GPU hardware "
              "accelerator.")
    if "kaggle_secrets" in sys.modules:
        print("Go to Settings > Accelerator and select GPU.")

No GPU was detected. Neural nets can be very slow without a GPU.


2023-05-21 22:09:09.285944: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcuda.so.1'; dlerror: libcuda.so.1: cannot open shared object file: No such file or directory
2023-05-21 22:09:09.295793: W tensorflow/compiler/xla/stream_executor/cuda/cuda_driver.cc:265] failed call to cuInit: UNKNOWN ERROR (303)
2023-05-21 22:09:09.296052: I tensorflow/compiler/xla/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (avtr): /proc/driver/nvidia/version does not exist


# Serving a TensorFlow Model

Let's start by deploying a model using TF Serving, then we'll deploy to Google Vertex AI.

## Using TensorFlow Serving

The first thing we need to do is to build and train a model, and export it to the SavedModel format.

### Exporting SavedModels

Let's load the MNIST dataset, scale it, and split it.

In [5]:
from pathlib import Path
import tensorflow as tf

# extra code – load and split the MNIST dataset
mnist = tf.keras.datasets.mnist.load_data()
(X_train_full, y_train_full), (X_test, y_test) = mnist
X_valid, X_train = X_train_full[:5000], X_train_full[5000:]
y_valid, y_train = y_train_full[:5000], y_train_full[5000:]

# extra code – build & train an MNIST model (also handles image preprocessing)
tf.random.set_seed(42)
tf.keras.backend.clear_session()
model = tf.keras.Sequential([
    tf.keras.layers.Flatten(input_shape=[28, 28], dtype=tf.uint8),
    tf.keras.layers.Rescaling(scale=1 / 255),
    tf.keras.layers.Dense(100, activation="relu"),
    tf.keras.layers.Dense(10, activation="softmax")
])
model.compile(loss="sparse_categorical_crossentropy",
              optimizer=tf.keras.optimizers.SGD(learning_rate=1e-2),
              metrics=["accuracy"])
model.fit(X_train, y_train, epochs=10, validation_data=(X_valid, y_valid))

model_name = "my_mnist_model"
model_version = "0001"
model_path = Path(model_name) / model_version
model.save(model_path, save_format="tf")

11490434/11490434 [==============================] - 7s 1us/step


2023-05-21 22:11:56.294548: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


Epoch 1/10
1719/1719 [==============================] - 18s 8ms/step - loss: 0.6984 - accuracy: 0.8221 - val_loss: 0.3734 - val_accuracy: 0.8996
Epoch 2/10
1719/1719 [==============================] - 11s 6ms/step - loss: 0.3513 - accuracy: 0.9021 - val_loss: 0.3010 - val_accuracy: 0.9140
Epoch 3/10
1719/1719 [==============================] - 11s 6ms/step - loss: 0.3000 - accuracy: 0.9149 - val_loss: 0.2653 - val_accuracy: 0.9282
Epoch 4/10
1719/1719 [==============================] - 8s 5ms/step - loss: 0.2689 - accuracy: 0.9241 - val_loss: 0.2426 - val_accuracy: 0.9338
Epoch 5/10
1719/1719 [==============================] - 7s 4ms/step - loss: 0.2457 - accuracy: 0.9307 - val_loss: 0.2238 - val_accuracy: 0.9374
Epoch 6/10
1719/1719 [==============================] - 6s 4ms/step - loss: 0.2272 - accuracy: 0.9359 - val_loss: 0.2102 - val_accuracy: 0.9422
Epoch 7/10
1719/1719 [==============================] - 7s 4ms/step - loss: 0.2117 - accuracy: 0.9405 - val_loss: 0.1953 - val_accura

INFO:tensorflow:Assets written to: my_mnist_model/0001/assets


INFO:tensorflow:Assets written to: my_mnist_model/0001/assets


Let's take a look at the file tree (we've discussed what each of these file is used for in chapter 10):

In [6]:
sorted([str(path) for path in model_path.parent.glob("**/*")])  # extra code

['my_mnist_model/0001',
 'my_mnist_model/0001/assets',
 'my_mnist_model/0001/fingerprint.pb',
 'my_mnist_model/0001/keras_metadata.pb',
 'my_mnist_model/0001/saved_model.pb',
 'my_mnist_model/0001/variables',
 'my_mnist_model/0001/variables/variables.data-00000-of-00001',
 'my_mnist_model/0001/variables/variables.index']

Let's inspect the SavedModel:

In [7]:
!saved_model_cli show --dir '{model_path}'

2023-05-21 22:13:31.910965: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-05-21 22:13:32.133569: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2023-05-21 22:13:32.133603: I tensorflow/compiler/xla/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.
2023-05-21 22:13:34.174534: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory
2023-

In [8]:
!saved_model_cli show --dir '{model_path}' --tag_set serve

2023-05-21 22:13:42.732275: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-05-21 22:13:42.949183: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2023-05-21 22:13:42.949217: I tensorflow/compiler/xla/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.
2023-05-21 22:13:44.392802: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory
2023-

In [9]:
!saved_model_cli show --dir '{model_path}' --tag_set serve \
                      --signature_def serving_default

2023-05-21 22:13:47.818497: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-05-21 22:13:48.114632: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2023-05-21 22:13:48.114666: I tensorflow/compiler/xla/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.
2023-05-21 22:13:49.377121: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory
2023-

For even more details, you can run the following command:

```ipython
!saved_model_cli show --dir '{model_path}' --all
```

###  Installing and Starting TensorFlow Serving

If you are running this notebook in Colab or Kaggle, TensorFlow Server needs to be installed:

In [10]:
if "google.colab" in sys.modules or "kaggle_secrets" in sys.modules:
    url = "https://storage.googleapis.com/tensorflow-serving-apt"
    src = "stable tensorflow-model-server tensorflow-model-server-universal"
    !echo 'deb {url} {src}' > /etc/apt/sources.list.d/tensorflow-serving.list
    !curl '{url}/tensorflow-serving.release.pub.gpg' | apt-key add -
    !apt update -q && apt-get install -y tensorflow-model-server
    %pip install -q -U tensorflow-serving-api

If `tensorflow_model_server` is installed (e.g., if you are running this notebook in Colab), then the following 2 cells will start the server. If your OS is Windows, you may need to run the `tensorflow_model_server` command in a terminal, and replace `${MODEL_DIR}` with the full path to the `my_mnist_model` directory.

In [11]:
import os

os.environ["MODEL_DIR"] = str(model_path.parent.absolute())

In [12]:
%%bash --bg
tensorflow_model_server \
    --port=8500 \
    --rest_api_port=8501 \
    --model_name=my_mnist_model \
    --model_base_path="${MODEL_DIR}" >my_server.log 2>&1

If you are running this notebook on your own machine, and you prefer to install TF Serving using Docker, first make sure [Docker](https://docs.docker.com/install/) is installed, then run the following commands in a terminal. You must replace `/path/to/my_mnist_model` with the appropriate absolute path to the `my_mnist_model` directory, but do not modify the container path `/models/my_mnist_model`.

```bash
docker pull tensorflow/serving  # downloads the latest TF Serving image

docker run -it --rm -v "/path/to/my_mnist_model:/models/my_mnist_model" \
    -p 8500:8500 -p 8501:8501 -e MODEL_NAME=my_mnist_model tensorflow/serving
```

### Querying TF Serving through the REST API

Next, let's send a REST query to TF Serving:

In [13]:
import json

X_new = X_test[:3]  # pretend we have 3 new digit images to classify
request_json = json.dumps({
    "signature_name": "serving_default",
    "instances": X_new.tolist(),
})

In [15]:
request_json[:100] + "..." + request_json[-10:]

'{"signature_name": "serving_default", "instances": [[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0..., 0, 0]]]}'

Now let's use TensorFlow Serving's REST API to make predictions:

In [ ]:
import requests

server_url = "http://localhost:8501/v1/models/my_mnist_model:predict"
response = requests.post(server_url, data=request_json)
response.raise_for_status()  # raise an exception in case of error
response = response.json()

In [ ]:
import numpy as np

y_proba = np.array(response["predictions"])
y_proba.round(2)

### Querying TF Serving through gRPC API

In [ ]:
from tensorflow_serving.apis.predict_pb2 import PredictRequest

request = PredictRequest()
request.model_spec.name = model_name
request.model_spec.signature_name = "serving_default"
input_name = model.input_names[0]  # == "flatten_input"
request.inputs[input_name].CopyFrom(tf.make_tensor_proto(X_new))

In [ ]:
import grpc
from tensorflow_serving.apis import prediction_service_pb2_grpc

channel = grpc.insecure_channel('localhost:8500')
predict_service = prediction_service_pb2_grpc.PredictionServiceStub(channel)
response = predict_service.Predict(request, timeout=10.0)

Convert the response to a tensor:

In [ ]:
output_name = model.output_names[0]
outputs_proto = response.outputs[output_name]
y_proba = tf.make_ndarray(outputs_proto)

In [ ]:
y_proba.round(2)

If your client does not include the TensorFlow library, you can convert the response to a NumPy array like this:

In [ ]:
# extra code – shows how to avoid using tf.make_ndarray()
output_name = model.output_names[0]
outputs_proto = response.outputs[output_name]
shape = [dim.size for dim in outputs_proto.tensor_shape.dim]
y_proba = np.array(outputs_proto.float_val).reshape(shape)
y_proba.round(2)

### Deploying a new model version

In [21]:
# extra code – build and train a new MNIST model version
np.random.seed(42)
tf.random.set_seed(42)
model = tf.keras.Sequential([
    tf.keras.layers.Flatten(input_shape=[28, 28], dtype=tf.uint8),
    tf.keras.layers.Rescaling(scale=1 / 255),
    tf.keras.layers.Dense(50, activation="relu"),
    tf.keras.layers.Dense(50, activation="relu"),
    tf.keras.layers.Dense(10, activation="softmax")
])
model.compile(loss="sparse_categorical_crossentropy",
              optimizer=tf.keras.optimizers.SGD(learning_rate=1e-2),
              metrics=["accuracy"])
history = model.fit(X_train, y_train, epochs=10,
                    validation_data=(X_valid, y_valid))

Epoch 1/10
1719/1719 [==============================] - 13s 6ms/step - loss: 0.7077 - accuracy: 0.8062 - val_loss: 0.3300 - val_accuracy: 0.9038
Epoch 2/10
1719/1719 [==============================] - 13s 7ms/step - loss: 0.3133 - accuracy: 0.9096 - val_loss: 0.2640 - val_accuracy: 0.9230
Epoch 3/10
1719/1719 [==============================] - 11s 7ms/step - loss: 0.2640 - accuracy: 0.9239 - val_loss: 0.2268 - val_accuracy: 0.9330
Epoch 4/10
1719/1719 [==============================] - 8s 4ms/step - loss: 0.2314 - accuracy: 0.9331 - val_loss: 0.2024 - val_accuracy: 0.9422
Epoch 5/10
1719/1719 [==============================] - 7s 4ms/step - loss: 0.2068 - accuracy: 0.9408 - val_loss: 0.1851 - val_accuracy: 0.9454
Epoch 6/10
1719/1719 [==============================] - 8s 5ms/step - loss: 0.1873 - accuracy: 0.9468 - val_loss: 0.1719 - val_accuracy: 0.9520
Epoch 7/10
1719/1719 [==============================] - 9s 5ms/step - loss: 0.1722 - accuracy: 0.9515 - val_loss: 0.1593 - val_accura

In [22]:
model_version = "0002"
model_path = Path(model_name) / model_version
model.save(model_path, save_format="tf")

INFO:tensorflow:Assets written to: my_mnist_model/0002/assets


INFO:tensorflow:Assets written to: my_mnist_model/0002/assets


Let's take a look at the file tree again:

In [23]:
sorted([str(path) for path in model_path.parent.glob("**/*")])  # extra code

['my_mnist_model/0001',
 'my_mnist_model/0001/assets',
 'my_mnist_model/0001/fingerprint.pb',
 'my_mnist_model/0001/keras_metadata.pb',
 'my_mnist_model/0001/saved_model.pb',
 'my_mnist_model/0001/variables',
 'my_mnist_model/0001/variables/variables.data-00000-of-00001',
 'my_mnist_model/0001/variables/variables.index',
 'my_mnist_model/0002',
 'my_mnist_model/0002/assets',
 'my_mnist_model/0002/fingerprint.pb',
 'my_mnist_model/0002/keras_metadata.pb',
 'my_mnist_model/0002/saved_model.pb',
 'my_mnist_model/0002/variables',
 'my_mnist_model/0002/variables/variables.data-00000-of-00001',
 'my_mnist_model/0002/variables/variables.index']

**Warning**: You may need to wait a minute before the new model is loaded by TensorFlow Serving.

In [ ]:
import requests

server_url = "http://localhost:8501/v1/models/my_mnist_model:predict"
            
response = requests.post(server_url, data=request_json)
response.raise_for_status()
response = response.json()

In [ ]:
response.keys()

In [ ]:
y_proba = np.array(response["predictions"])
y_proba.round(2)

## Creating a Prediction Service on Vertex AI

Follow the instructions in the book to create a Google Cloud Platform account and activate the Vertex AI and Cloud Storage APIs. Then, if you're running this notebook in Colab, you can run the following cell to authenticate using the same Google account as you used with Google Cloud Platform, and authorize this Colab to access your data.

**WARNING: only do this if you trust this notebook!**
* Be extra careful if this is not the official notebook from https://github.com/ageron/handson-ml3: the Colab URL should start with https://colab.research.google.com/github/ageron/handson-ml3. Or else, the code could do whatever it wants with your data.

If you are not running this notebook in Colab, you must follow the instructions in the book to create a service account and generate a key for it, download it to this notebook's directory, and name it `my_service_account_key.json` (or make sure the `GOOGLE_APPLICATION_CREDENTIALS` environment variable points to your key).

In [ ]:
project_id = "my_project"  ##### CHANGE THIS TO YOUR PROJECT ID #####

if "google.colab" in sys.modules:
    from google.colab import auth
    auth.authenticate_user()
elif "kaggle_secrets" in sys.modules:
    from kaggle_secrets import UserSecretsClient
    UserSecretsClient().set_gcloud_credentials(project=project_id)
else:
    os.environ["GOOGLE_APPLICATION_CREDENTIALS"] = "my_service_account_key.json"

In [ ]:
from google.cloud import storage

bucket_name = "my_bucket"  ##### CHANGE THIS TO A UNIQUE BUCKET NAME #####
location = "us-central1"

storage_client = storage.Client(project=project_id)
bucket = storage_client.create_bucket(bucket_name, location=location)
#bucket = storage_client.bucket(bucket_name)  # to reuse a bucket instead

In [ ]:
def upload_directory(bucket, dirpath):
    dirpath = Path(dirpath)
    for filepath in dirpath.glob("**/*"):
        if filepath.is_file():
            blob = bucket.blob(filepath.relative_to(dirpath.parent).as_posix())
            blob.upload_from_filename(filepath)

upload_directory(bucket, "my_mnist_model")

In [ ]:
# extra code – a much faster multithreaded implementation of upload_directory()
#              which also accepts a prefix for the target path, and prints stuff

from concurrent import futures

def upload_file(bucket, filepath, blob_path):
    blob = bucket.blob(blob_path)
    blob.upload_from_filename(filepath)

def upload_directory(bucket, dirpath, prefix=None, max_workers=50):
    dirpath = Path(dirpath)
    prefix = prefix or dirpath.name
    with futures.ThreadPoolExecutor(max_workers=max_workers) as executor:
        future_to_filepath = {
            executor.submit(
                upload_file,
                bucket, filepath,
                f"{prefix}/{filepath.relative_to(dirpath).as_posix()}"
            ): filepath
            for filepath in sorted(dirpath.glob("**/*"))
            if filepath.is_file()
        }
        for future in futures.as_completed(future_to_filepath):
            filepath = future_to_filepath[future]
            try:
                result = future.result()
            except Exception as ex:
                print(f"Error uploading {filepath!s:60}: {ex}")  # f!s is str(f)
            else:
                print(f"Uploaded {filepath!s:60}", end="\r")

    print(f"Uploaded {dirpath!s:60}")

Alternatively, if you installed Google Cloud CLI (it's preinstalled on Colab), then you can use the following `gsutil` command:

In [25]:
#!gsutil -m cp -r my_mnist_model gs://{bucket_name}/

In [ ]:
from google.cloud import aiplatform

server_image = "gcr.io/cloud-aiplatform/prediction/tf2-gpu.2-8:latest"

aiplatform.init(project=project_id, location=location)
mnist_model = aiplatform.Model.upload(
    display_name="mnist",
    artifact_uri=f"gs://{bucket_name}/my_mnist_model/0001",
    serving_container_image_uri=server_image,
)

**Warning**: this cell may take several minutes to run, as it waits for Vertex AI to provision the compute nodes:

In [ ]:
endpoint = aiplatform.Endpoint.create(display_name="mnist-endpoint")

endpoint.deploy(
    mnist_model,
    min_replica_count=1,
    max_replica_count=5,
    machine_type="n1-standard-4",
    accelerator_type="NVIDIA_TESLA_K80",
    accelerator_count=1
)

In [ ]:
response = endpoint.predict(instances=X_new.tolist())

In [ ]:
import numpy as np

np.round(response.predictions, 2)

In [ ]:
endpoint.undeploy_all()  # undeploy all models from the endpoint
endpoint.delete()

## Running Batch Prediction Jobs on Vertex AI

batch_path = Path("my_mnist_batch")
batch_path.mkdir(exist_ok=True)
with open(batch_path / "my_mnist_batch.jsonl", "w") as jsonl_file:
    for image in X_test[:100].tolist():
        jsonl_file.write(json.dumps(image))
        jsonl_file.write("\n")

upload_directory(bucket, batch_path)

In [ ]:
batch_prediction_job = mnist_model.batch_predict(
    job_display_name="my_batch_prediction_job",
    machine_type="n1-standard-4",
    starting_replica_count=1,
    max_replica_count=5,
    accelerator_type="NVIDIA_TESLA_K80",
    accelerator_count=1,
    gcs_source=[f"gs://{bucket_name}/{batch_path.name}/my_mnist_batch.jsonl"],
    gcs_destination_prefix=f"gs://{bucket_name}/my_mnist_predictions/",
    sync=True  # set to False if you don't want to wait for completion
)

In [ ]:
batch_prediction_job.output_info  # extra code – shows the output directory

In [ ]:
y_probas = []
for blob in batch_prediction_job.iter_outputs():
    print(blob.name)  # extra code
    if "prediction.results" in blob.name:
        for line in blob.download_as_text().splitlines():
            y_proba = json.loads(line)["prediction"]
            y_probas.append(y_proba)

In [ ]:
y_pred = np.argmax(y_probas, axis=1)
accuracy = np.sum(y_pred == y_test[:100]) / 100

In [ ]:
accuracy

In [ ]:
mnist_model.delete()

Let's delete all the directories we created on GCS (i.e., all the blobs with these prefixes):

In [ ]:
for prefix in ["my_mnist_model/", "my_mnist_batch/", "my_mnist_predictions/"]:
    blobs = bucket.list_blobs(prefix=prefix)
    for blob in blobs:
        blob.delete()

#bucket.delete()  # uncomment and run if you want to delete the bucket itself
batch_prediction_job.delete()

# Deploying a Model to a Mobile or Embedded Device

In [ ]:
converter = tf.lite.TFLiteConverter.from_saved_model(str(model_path))
tflite_model = converter.convert()
with open("my_converted_savedmodel.tflite", "wb") as f:
    f.write(tflite_model)

In [ ]:
# extra code – shows how to convert a Keras model
converter = tf.lite.TFLiteConverter.from_keras_model(model)

In [ ]:
converter.optimizations = [tf.lite.Optimize.DEFAULT]

In [ ]:
tflite_model = converter.convert()
with open("my_converted_keras_model.tflite", "wb") as f:
    f.write(tflite_model)

# Running a Model in a Web Page

Code examples for this section are hosted on glitch.com, a website that lets you create Web apps for free.

* https://homl.info/tfjscode: a simple TFJS Web app that loads a pretrained model and classifies an image.
* https://homl.info/tfjswpa: the same Web app setup as a WPA. Try opening this link on various platforms, including mobile devices.
** https://homl.info/wpacode: this WPA's source code.
* https://tensorflow.org/js: The TFJS library.
** https://www.tensorflow.org/js/demos: some fun demos.

# Using GPUs to Speed Up Computations

Let's check that TensorFlow can see the GPU:

In [26]:
physical_gpus = tf.config.list_physical_devices("GPU")
physical_gpus

[]

If you want your TensorFlow script to use only GPUs \#0 and \#1 (based on PCI order), then you can set the environment variables `CUDA_DEVICE_ORDER=PCI_BUS_ID` and `CUDA_VISIBLE_DEVICES=0,1` before starting your script, or in the script itself before using TensorFlow.